In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from pyts.image import GramianAngularField
from pyts.datasets import load_gunpoint
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import resample
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks, NearMiss
from sklearn.model_selection import train_test_split
import h5py
from PIL import Image
from torchvision import datasets, transforms

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

import torch.nn as nn
from torchvision.models import alexnet, vgg16, resnet152, resnet18, vgg19
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report, recall_score, f1_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [4]:
ckpt_gaf = torch.load('res_net_test_recall_best_gaf.chk')
res_net_gaf = resnet18(pretrained=True)
for param in res_net_gaf.parameters():
    param.requires_grad = False

num_ftrs = res_net_gaf.fc.in_features
res_net_gaf.fc = nn.Sequential(
                nn.Linear(in_features=num_ftrs, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
                nn.Linear(in_features=128, out_features=5, bias=True))
res_net_gaf.load_state_dict(ckpt_gaf['net'])
res_net_gaf.fc = Identity()
res_net_gaf.cuda()   

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [5]:
ckpt_rp = torch.load('res_net_test_recall_best_rp.chk')
res_net_rp = resnet18(pretrained=True)
for param in res_net_rp.parameters():
    param.requires_grad = False

num_ftrs = res_net_rp.fc.in_features
res_net_rp.fc = nn.Sequential(
                nn.Linear(in_features=num_ftrs, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
                nn.Linear(in_features=128, out_features=5, bias=True))
res_net_rp.load_state_dict(ckpt_rp['net'])
res_net_rp.fc = Identity()
res_net_rp.cuda()   

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [8]:
# Function for moving tensor or model to GPU
def cuda(xs):
    if torch.cuda.is_available():
        if not isinstance(xs, (list, tuple)):
            return xs.cuda()
        else:
            return [x.cuda() for x in xs]
    else:
        return xs

# Custom class for defining dataset for training with augmentation
class Dataset_Hdf5(Dataset):

    def __init__(self, path, data_type):
        """ Intialize the dataset
        """
        self.path = path
        self.file = h5py.File(path, 'r')
        self.images = self.file['x_{}'.format(data_type)]
        self.labels = self.file['y_{}'.format(data_type)]
                
        self.len = self.images.shape[0]
        if data_type == 'train':
            # no augmentation as we are transforming heartbeats to images via gramian angular field
            self.transform = transforms.Compose([
#                                               transforms.ToPILImage(),
#                                               transforms.RandomRotation((0, 360)),
#                                               transforms.RandomHorizontalFlip(),
#                                               transforms.RandomVerticalFlip(),
                                              transforms.ToTensor()])
        else:
            self.transform = transforms.Compose([transforms.ToTensor()])

    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        # unsqueeze adds dimension to image -> converts to 1x224x224 since we don't have rgb
        return self.transform(self.images[index].astype('float32')), \
                torch.tensor(self.labels[index], dtype=torch.long)

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

In [9]:
hb_train_loader_bln_3 = torch.utils.data.DataLoader(Dataset_Hdf5('/home/asif/heartbeat/hb_data_ptb.hdf5', 'train'), 
                                                batch_size=512, shuffle=True)
hb_test_loader_bln_3 = torch.utils.data.DataLoader(Dataset_Hdf5('/home/asif/heartbeat/hb_data_ptb.hdf5', 'test'), 
                                                batch_size=512, shuffle=False)

In [10]:
hb_train_loader_bln_4 = torch.utils.data.DataLoader(Dataset_Hdf5('/home/asif/heartbeat/hb_data_ptb_rp.hdf5', 'train'), 
                                                batch_size=512, shuffle=False)
hb_test_loader_bln_4 = torch.utils.data.DataLoader(Dataset_Hdf5('/home/asif/heartbeat/hb_data_ptb_rp.hdf5', 'test'), 
                                                batch_size=512, shuffle=False)

In [14]:
train_size = 11641
test_size = 2911

In [15]:
with h5py.File('hb_gaf_rp_ptb_intermediate.hdf5', mode='w') as hdf5_file:
    hdf5_file.create_dataset("x_train", (train_size, 1024), np.float32)
    hdf5_file.create_dataset("y_train", (train_size,), np.int32)
    hdf5_file.create_dataset("x_test", (test_size, 1024), np.float32)
    hdf5_file.create_dataset("y_test", (test_size,), np.int32)
    
    index = 0
    for _, (data_gaf, data_rp) in enumerate(zip(hb_train_loader_bln_3, hb_train_loader_bln_4), 0):
        inputs_gaf, labels = cuda(data_gaf)
        inputs_rp, _ = cuda(data_rp)
        outputs_gaf = res_net_gaf(inputs_gaf.expand(-1, 3, -1, -1))
        outputs_rp = res_net_rp(inputs_rp.expand(-1, 3, -1, -1))
        hdf5_file["x_train"][index:index + outputs_gaf.shape[0], :512] = outputs_gaf.cpu().numpy()
        hdf5_file["x_train"][index:index + outputs_rp.shape[0], 512:1024] = outputs_rp.cpu().numpy()
        hdf5_file["y_train"][index:index + outputs_gaf.shape[0]] = labels.cpu().numpy()
        index += outputs_gaf.shape[0]
    print(index)
    index = 0
    for _, (data_gaf, data_rp) in enumerate(zip(hb_test_loader_bln_3, hb_test_loader_bln_4), 0):
        inputs_gaf, labels = cuda(data_gaf)
        inputs_rp, _ = cuda(data_rp)
        outputs_gaf = res_net_gaf(inputs_gaf.expand(-1, 3, -1, -1))
        outputs_rp = res_net_rp(inputs_rp.expand(-1, 3, -1, -1))
        hdf5_file["x_test"][index:index + outputs_gaf.shape[0], :512] = outputs_gaf.cpu().numpy()
        hdf5_file["x_test"][index:index + outputs_rp.shape[0], 512:1024] = outputs_rp.cpu().numpy()
        hdf5_file["y_test"][index:index + outputs_gaf.shape[0]] = labels.cpu().numpy()
        index += outputs_gaf.shape[0]

11641


In [30]:
net = nn.Sequential(
                nn.Linear(in_features=1024, out_features=512, bias=False),
                nn.ReLU(),
#                 nn.Dropout(p=0.5),
                nn.Linear(in_features=512, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
#                 nn.Dropout(p=0.5),
                nn.Linear(in_features=128, out_features=2, bias=True))
net.cuda()

Sequential(
  (0): Linear(in_features=1024, out_features=512, bias=False)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=256, bias=False)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=2, bias=True)
)

In [31]:
# Custom class for defining dataset for training with augmentation
class DatasetFusion(Dataset):

    def __init__(self, path, data_type):
        """ Intialize the dataset
        """
        self.path = path
        self.file = h5py.File(path, 'r')
        self.images = self.file['x_{}'.format(data_type)]
        self.labels = self.file['y_{}'.format(data_type)]
                
        self.len = self.images.shape[0]
        if data_type == 'train':
            # no augmentation as we are transforming heartbeats to images via gramian angular field
            self.transform = transforms.Compose([

                                              transforms.ToTensor()])
        else:
            self.transform = transforms.Compose([transforms.ToTensor()])

    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        # unsqueeze adds dimension to image -> converts to 1x224x224 since we don't have rgb
        return torch.tensor(self.images[index].astype('float32')), \
                torch.tensor(self.labels[index], dtype=torch.long)

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

In [32]:
hb_train_loader = torch.utils.data.DataLoader(DatasetFusion('/home/asif/heartbeat/hb_gaf_rp_ptb_intermediate.hdf5', 'train'), 
                                                batch_size=512, shuffle=True)
hb_test_loader = torch.utils.data.DataLoader(DatasetFusion('/home/asif/heartbeat/hb_gaf_rp_ptb_intermediate.hdf5', 'test'), 
                                                batch_size=512, shuffle=False)

In [33]:
class_weights = cuda(torch.tensor([2.0, 1.0]))
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [34]:
optimizer = torch.optim.Adam([
                                          {"params": net[0].parameters(), "lr": 0.001},
                                          {"params": net[2].parameters(), "lr": 0.001},
                                          {"params": net[4].parameters(), "lr": 0.002},
#                                           {"params": net[6].parameters(), "lr": 0.002},
                                           ],
                                lr=0.001, betas=(0.5, 0.999))

In [35]:
def train(net, train_loader, criterion, optimizer, test_loader, num_epochs=25):
    net.train()
    train_acc_max = 0
    test_acc_max = 0
    recall_max = 0
    f1_max = 0

#     scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
    
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        
        net.train()

        total = 0
        correct = 0
   
        running_loss = 0.0
    
        for i, data in enumerate(train_loader, 0):
            
            # get the inputs; data is a list of [inputs, labels]
            
            inputs, labels = cuda(data)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs) # already in RGB
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        
#         scheduler.step()
        
        print('End of epoch {}, Loss {}'.format(epoch + 1, running_loss / len(train_loader)))
        
        train_acc = correct / total
        print('Train accuracy: {}'.format(train_acc))
        test_acc, all_true, all_pred = test(net, test_loader)
        print('Test accuracy: {}'.format(test_acc))
        print(classification_report(all_true, all_pred, target_names=['N', 'A']))
        recall = recall_score(all_true, all_pred, average='macro')
        f1 = f1_score(all_true, all_pred, average='macro')
        
        # Saving best checkpoint based on performance on test data            
        if recall > recall_max:
            recall_max = recall
            save_checkpoint(epoch + 1, net, optimizer, train_acc, test_acc, recall, f1, 'fc_net_fusion', 'test_recall')
            
        if f1 > f1_max:
            f1_max = f1
            save_checkpoint(epoch + 1, net, optimizer, train_acc, test_acc, recall, f1, 'fc_net_fusion', 'test_f1')

    print('Finished Training')
    
def test(net, test_loader):
    net.eval()
    correct = 0
    total = 0
    all_true = []
    all_pred = []
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            inputs, labels = cuda(data)
            all_true.extend(labels.cpu().tolist())
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            all_pred.extend(predicted.cpu().tolist())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = correct / total
#     print('Accuracy of the network on the images: %d %%' % (100 * acc))
    return acc, all_true, all_pred

def save_checkpoint(epoch, net, optimizer, train_acc, test_acc, recall, f1, net_name, param):
    checkpoint = {
        'net': net.state_dict(),
        'optimizer': optimizer.state_dict(),
        'train_acc': train_acc,
        'test_acc': test_acc,
        'recall': recall,
        'f1': f1
    }
    torch.save(checkpoint, '{}_{}_best.chk'.format(net_name, param))

In [37]:
train(net, hb_train_loader, criterion, optimizer, hb_test_loader, 50)

End of epoch 1, Loss 0.2868299581434416
Train accuracy: 0.8842882913839017
Test accuracy: 0.8450704225352113
              precision    recall  f1-score   support

           N       0.66      0.91      0.76       809
           A       0.96      0.82      0.88      2102

    accuracy                           0.85      2911
   macro avg       0.81      0.86      0.82      2911
weighted avg       0.88      0.85      0.85      2911

End of epoch 2, Loss 0.2805394109176553
Train accuracy: 0.8836869684734988
Test accuracy: 0.889385091034009
              precision    recall  f1-score   support

           N       0.79      0.83      0.81       809
           A       0.93      0.91      0.92      2102

    accuracy                           0.89      2911
   macro avg       0.86      0.87      0.86      2911
weighted avg       0.89      0.89      0.89      2911

End of epoch 3, Loss 0.27650885867035907
Train accuracy: 0.8853191306588781
Test accuracy: 0.8849192717279285
              preci

End of epoch 20, Loss 0.2434803992509842
Train accuracy: 0.8980328150502535
Test accuracy: 0.8526279628993473
              precision    recall  f1-score   support

           N       0.68      0.90      0.77       809
           A       0.95      0.84      0.89      2102

    accuracy                           0.85      2911
   macro avg       0.82      0.87      0.83      2911
weighted avg       0.88      0.85      0.86      2911

End of epoch 21, Loss 0.19546976685523987
Train accuracy: 0.9171892449102311
Test accuracy: 0.8790793541738234
              precision    recall  f1-score   support

           N       0.73      0.89      0.80       809
           A       0.95      0.88      0.91      2102

    accuracy                           0.88      2911
   macro avg       0.84      0.88      0.86      2911
weighted avg       0.89      0.88      0.88      2911

End of epoch 22, Loss 0.2267635393401851
Train accuracy: 0.8999226870543767
Test accuracy: 0.8880109927859842
              p

End of epoch 39, Loss 0.1487905373391898
Train accuracy: 0.9390086762305644
Test accuracy: 0.8488491927172793
              precision    recall  f1-score   support

           N       0.92      0.50      0.65       809
           A       0.84      0.98      0.90      2102

    accuracy                           0.85      2911
   macro avg       0.88      0.74      0.78      2911
weighted avg       0.86      0.85      0.83      2911

End of epoch 40, Loss 0.24350104254225027
Train accuracy: 0.8982046215960828
Test accuracy: 0.8856063208519409
              precision    recall  f1-score   support

           N       0.76      0.85      0.81       809
           A       0.94      0.90      0.92      2102

    accuracy                           0.89      2911
   macro avg       0.85      0.88      0.86      2911
weighted avg       0.89      0.89      0.89      2911

End of epoch 41, Loss 0.16063647101754727
Train accuracy: 0.9344558027660854
Test accuracy: 0.8873239436619719
              